In [1]:
import malariagen_data
import allel

/home/chris/.cache/pypoetry/virtualenvs/malariagen-data-mfr76-fD-py3.6/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%run "../malariagen_data/util.py"

In [3]:
ag3 = malariagen_data.Ag3()

In [5]:
contig_dict = {
    "2R": 0, 
    "2L": 1,
    "3R": 2,
    "3L": 3,
    "X" : 4
}

In [8]:
aims = "gamb_vs_colu"
sample_sets = "AG1000G-UG"

In [41]:
def aim_calls(aims, sample_sets, sample_query=None):
    
    # contig dict
    contig_dict = {
    "2R": 0, 
#     "2L": 1,
#     "3R": 2,
#     "3L": 3,
#     "X" : 4
    }
    
    # output dict
    ds_dict = {}
    
    aim_sites = ag3.aim_sites(aims=aims)
    aim_sites = aim_sites.set_index(variants='variant_contig')
    
    # get calls for each aim
    for c in contig_dict:
        print(f"collecting {c} aim calls")
        # aim alleles and positions
        ca_sites = aim_sites.sel(variants=contig_dict[c])
        ref = ca_sites['variant_allele'][:,0].values
        alt = ca_sites['variant_allele'][:,1].values
        ca_pos = ca_sites['variant_position'].values
        # snps at aim positions
        snp_calls = ag3.snp_calls(region=c, sample_sets=sample_sets)
        ca_calls = snp_calls.set_index(variants='variant_position')
        sel_calls = ca_calls.sel(variants=ca_pos)
        alleles = sel_calls['variant_allele'][:].values
        # map aim/snp alleles
        mapping = allel.create_allele_mapping(ref, alt, alleles)
        gt = allel.GenotypeDaskArray(sel_calls['call_genotype'].data)
        gt_map = gt.map_alleles(mapping)
        
#         ds_dict[c] = ca_calls.sel(variants=ca_pos)
        
#     # concat at this point?    
#     ds_concat = xarray_concat([ds_dict[c] for c in contig_dict], "variants")
        
    return ca_sites, sel_calls

In [42]:
%%time
ca_sites, sel_calls = aim_calls(aims=aims, sample_sets=sample_sets)

collecting 2R aim calls
CPU times: user 6.85 s, sys: 3.14 s, total: 9.99 s
Wall time: 29.5 s


In [51]:
ref = ca_sites['variant_allele'][:,0].values

In [52]:
alt = ca_sites['variant_allele'][:,1].values

In [60]:
alleles = sel_calls['variant_allele'][:].values

In [63]:
ref

array([b'A', b'C', b'G', b'T', b'G', b'G', b'G', b'C', b'T', b'G', b'C',
       b'G', b'T', b'C', b'A', b'G', b'G', b'T', b'G', b'T', b'G', b'T',
       b'T', b'A', b'C', b'G', b'T', b'T', b'C', b'C', b'C', b'A', b'T',
       b'A', b'G', b'C', b'T', b'C', b'T', b'T', b'G', b'A', b'T', b'G',
       b'A', b'G', b'C', b'T', b'C', b'T', b'G', b'C', b'A', b'C', b'T',
       b'T', b'T', b'T'], dtype='|S1')

In [64]:
alt

array([b'G', b'A', b'A', b'A', b'A', b'A', b'A', b'A', b'G', b'A', b'A',
       b'A', b'C', b'T', b'T', b'A', b'A', b'A', b'C', b'C', b'A', b'C',
       b'G', b'G', b'A', b'C', b'A', b'A', b'T', b'T', b'T', b'T', b'A',
       b'G', b'A', b'A', b'C', b'A', b'C', b'G', b'A', b'G', b'A', b'T',
       b'T', b'A', b'G', b'G', b'T', b'G', b'C', b'A', b'G', b'T', b'A',
       b'C', b'C', b'A'], dtype='|S1')

In [65]:
alleles

array([[b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'G', b'A', b'C', b'T'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'G', b'A', b'C', b'T'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'

In [58]:
len(alt)

58

In [61]:
mapping = allel.create_allele_mapping(ref, alt, alleles)

In [62]:
mapping

array([[0, 3],
       [1, 0],
       [3, 0],
       [2, 0],
       [3, 0],
       [3, 0],
       [3, 0],
       [1, 0],
       [3, 0],
       [3, 0],
       [1, 0],
       [3, 0],
       [2, 0],
       [0, 2],
       [0, 2],
       [3, 0],
       [3, 0],
       [2, 0],
       [3, 0],
       [2, 0],
       [3, 0],
       [2, 0],
       [3, 0],
       [0, 3],
       [1, 0],
       [3, 0],
       [2, 0],
       [2, 0],
       [0, 2],
       [0, 2],
       [0, 2],
       [0, 2],
       [2, 0],
       [0, 3],
       [3, 0],
       [1, 0],
       [2, 0],
       [1, 0],
       [2, 0],
       [3, 0],
       [3, 0],
       [0, 3],
       [2, 0],
       [0, 3],
       [0, 2],
       [3, 0],
       [0, 3],
       [3, 0],
       [0, 2],
       [3, 0],
       [3, 0],
       [1, 0],
       [0, 3],
       [0, 2],
       [2, 0],
       [2, 0],
       [2, 0],
       [2, 0]], dtype=int8)

In [69]:
gt = allel.GenotypeDaskArray(sel_calls['call_genotype'].data)

In [76]:
gt[:10,:10]

<GenotypeDaskArray shape=(10, 10, 2) dtype=int8>

In [72]:
gt_map = gt.map_alleles(mapping)

In [75]:
gt_map[:10,:10]

<GenotypeDaskArray shape=(10, 10, 2) dtype=int8>

In [31]:
snp_calls = ag3.snp_calls(region='2R', sample_sets="3.0")

In [35]:
snp_calls['variant_allele'][:1,:5].values

array([[b'C', b'A', b'T', b'G']], dtype='|S1')

In [4]:
aims = "gamb_vs_colu"
sample_sets = "AG1000G-UG"

In [5]:
aim_sites = ag3.aim_sites(aims=aims)

In [6]:
aim_sites

<xarray.Dataset>
Dimensions:           (alleles: 2, variants: 729)
Coordinates:
    variant_contig    (variants) uint8 dask.array<chunksize=(729,), meta=np.ndarray>
    variant_position  (variants) int64 dask.array<chunksize=(729,), meta=np.ndarray>
Dimensions without coordinates: alleles, variants
Data variables:
    variant_allele    (variants, alleles) |S1 dask.array<chunksize=(729, 2), meta=np.ndarray>
Attributes:
    contigs:  ['2R', '2L', '3R', '3L', 'X']

In [20]:
aim_sites = aim_sites.set_index(variants='variant_contig')

In [13]:
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0
Results cache           : None
Cohorts analysis        : 20211101
Species analysis        : aim_20200422
Site filters analysis   : dt_20200416
Software version        : malariagen_data 0.7.0
Client location         : England, GB
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

In [28]:
zero = aim_sites.sel(variants=0)

In [29]:
zero_pos = zero['variant_position'].values

In [8]:
region = '2R'

In [9]:
snp_calls = ag3.snp_calls(region=region, sample_sets="3.0")

In [10]:
snp_calls

<xarray.Dataset>
Dimensions:                             (alleles: 4, ploidy: 2, samples: 3081, variants: 60132453)
Coordinates:
    variant_position                    (variants) int32 dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_contig                      (variants) uint8 dask.array<chunksize=(524288,), meta=np.ndarray>
    sample_id                           (samples) <U24 dask.array<chunksize=(81,), meta=np.ndarray>
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    variant_allele                      (variants, alleles) |S1 dask.array<chunksize=(524288, 1), meta=np.ndarray>
    variant_filter_pass_gamb_colu_arab  (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    variant_filter_pass_gamb_colu       (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    variant_filter_pass_arab            (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    call_genotype                       (variants, samples, ploidy) int8 dask.array<chunksize=(300000, 50, 2), meta=np.ndarray>
    call_GQ                             (variants, samples) int16 dask.array<chunksize=(300000, 50), meta=np.ndarray>
    call_MQ                             (variants, samples) int16 dask.array<chunksize=(300000, 50), meta=np.ndarray>
    call_AD                             (variants, samples, alleles) int16 dask.array<chunksize=(300000, 50, 4), meta=np.ndarray>
    call_genotype_mask                  (variants, samples, ploidy) bool dask.array<chunksize=(300000, 50, 2), meta=np.ndarray>

In [12]:
snp_calls = snp_calls.set_index(variants='variant_position')

In [14]:
ds_dict= {}

In [30]:
%%time
ds_dict[0] = snp_calls.sel(variants=zero_pos)
ds_dict[0]

CPU times: user 611 ms, sys: 804 ms, total: 1.42 s
Wall time: 1.45 s


<xarray.Dataset>
Dimensions:                             (alleles: 4, ploidy: 2, samples: 3081, variants: 58)
Coordinates:
    variant_contig                      (variants) uint8 dask.array<chunksize=(1,), meta=np.ndarray>
    sample_id                           (samples) <U24 dask.array<chunksize=(81,), meta=np.ndarray>
  * variants                            (variants) int64 5896279 ... 61455401
Dimensions without coordinates: alleles, ploidy, samples
Data variables:
    variant_allele                      (variants, alleles) |S1 dask.array<chunksize=(1, 1), meta=np.ndarray>
    variant_filter_pass_gamb_colu_arab  (variants) bool dask.array<chunksize=(1,), meta=np.ndarray>
    variant_filter_pass_gamb_colu       (variants) bool dask.array<chunksize=(1,), meta=np.ndarray>
    variant_filter_pass_arab            (variants) bool dask.array<chunksize=(1,), meta=np.ndarray>
    call_genotype                       (variants, samples, ploidy) int8 dask.array<chunksize=(1, 50, 2), meta=np.ndarray>
    call_GQ                             (variants, samples) int16 dask.array<chunksize=(1, 50), meta=np.ndarray>
    call_MQ                             (variants, samples) int16 dask.array<chunksize=(1, 50), meta=np.ndarray>
    call_AD                             (variants, samples, alleles) int16 dask.array<chunksize=(1, 50, 4), meta=np.ndarray>
    call_genotype_mask                  (variants, samples, ploidy) bool dask.array<chunksize=(1, 50, 2), meta=np.ndarray>

In [15]:
%%time
one = aim_sites.sel(variants=1)
one_pos = one['variant_position'].values
region = '2L'
snp_calls = ag3.snp_calls(region=region, sample_sets="3.0")
snp_calls = snp_calls.set_index(variants='variant_position')

CPU times: user 1.81 s, sys: 319 ms, total: 2.13 s
Wall time: 28.9 s


In [16]:
snp_calls = ag3.snp_calls(region=region, sample_sets="3.0")

In [17]:
%%time
snp_calls = snp_calls.set_index(variants='variant_position')

CPU times: user 1.02 s, sys: 258 ms, total: 1.27 s
Wall time: 18.8 s


In [33]:
%%time
ds_dict[1] = snp_calls.sel(variants=one_pos)
ds_dict[1]

CPU times: user 513 ms, sys: 481 ms, total: 994 ms
Wall time: 1 s


<xarray.Dataset>
Dimensions:                             (alleles: 4, ploidy: 2, samples: 3081, variants: 95)
Coordinates:
    variant_contig                      (variants) uint8 dask.array<chunksize=(6,), meta=np.ndarray>
    sample_id                           (samples) <U24 dask.array<chunksize=(81,), meta=np.ndarray>
  * variants                            (variants) int64 181564 ... 11870350
Dimensions without coordinates: alleles, ploidy, samples
Data variables:
    variant_allele                      (variants, alleles) |S1 dask.array<chunksize=(6, 1), meta=np.ndarray>
    variant_filter_pass_gamb_colu_arab  (variants) bool dask.array<chunksize=(3,), meta=np.ndarray>
    variant_filter_pass_gamb_colu       (variants) bool dask.array<chunksize=(3,), meta=np.ndarray>
    variant_filter_pass_arab            (variants) bool dask.array<chunksize=(3,), meta=np.ndarray>
    call_genotype                       (variants, samples, ploidy) int8 dask.array<chunksize=(3, 50, 2), meta=np.ndarray>
    call_GQ                             (variants, samples) int16 dask.array<chunksize=(3, 50), meta=np.ndarray>
    call_MQ                             (variants, samples) int16 dask.array<chunksize=(3, 50), meta=np.ndarray>
    call_AD                             (variants, samples, alleles) int16 dask.array<chunksize=(3, 50, 4), meta=np.ndarray>
    call_genotype_mask                  (variants, samples, ploidy) bool dask.array<chunksize=(3, 50, 2), meta=np.ndarray>

In [43]:
test_concat = xarray_concat([ds_dict[0], ds_dict[1]], "variants")

In [44]:
test_concat

<xarray.Dataset>
Dimensions:                             (alleles: 4, ploidy: 2, samples: 3081, variants: 153)
Coordinates:
    variant_contig                      (variants) uint8 dask.array<chunksize=(1,), meta=np.ndarray>
    sample_id                           (samples) <U24 dask.array<chunksize=(81,), meta=np.ndarray>
  * variants                            (variants) int64 5896279 ... 11870350
Dimensions without coordinates: alleles, ploidy, samples
Data variables:
    variant_allele                      (variants, alleles) |S1 dask.array<chunksize=(1, 1), meta=np.ndarray>
    variant_filter_pass_gamb_colu_arab  (variants) bool dask.array<chunksize=(1,), meta=np.ndarray>
    variant_filter_pass_gamb_colu       (variants) bool dask.array<chunksize=(1,), meta=np.ndarray>
    variant_filter_pass_arab            (variants) bool dask.array<chunksize=(1,), meta=np.ndarray>
    call_genotype                       (variants, samples, ploidy) int8 dask.array<chunksize=(1, 50, 2), meta=np.ndarray>
    call_GQ                             (variants, samples) int16 dask.array<chunksize=(1, 50), meta=np.ndarray>
    call_MQ                             (variants, samples) int16 dask.array<chunksize=(1, 50), meta=np.ndarray>
    call_AD                             (variants, samples, alleles) int16 dask.array<chunksize=(1, 50, 4), meta=np.ndarray>
    call_genotype_mask                  (variants, samples, ploidy) bool dask.array<chunksize=(1, 50, 2), meta=np.ndarray>

In [50]:
test_concat["call_genotype"][:1].values

array([[[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 3],
        [0, 3],
        [0, 3]]], dtype=int8)

In [57]:
for c in contig_dict.keys():
    print(c)
    

2R
2L
3R
3L
X


In [53]:
contig_dict = {
    "2R": 0, 
    "2L": 1,
    "3R": 2,
    "3L": 3,
    "X" : 4
}

In [ ]:
snp_calls.sel?